In [36]:
# import Required libraries
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

import configparser

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
!pip  install autocorrect
# from autocorrect import Speller



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
config = configparser.ConfigParser()

# Load the data

# from google.colab import files
# uploaded = files.upload()

from google.colab import drive
drive.mount('/content/drive/')
config.read("drive/My Drive/Colab Notebooks/config.ini")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


['drive/My Drive/Colab Notebooks/config.ini']

In [38]:
# Twitter API credentials
api_key = config['twitter']['Api_key']
api_key_secret = config['twitter']['Api_key_secret']

access_token = config['twitter']['Access_token']
access_token_secret = config['twitter']['Access_token_secret']

In [39]:
#  Create authentication object
auth = tweepy.OAuthHandler(api_key, api_key_secret)

# Set the access token and access token secret
auth.set_access_token(access_token, access_token_secret)

# Create the API object
api = tweepy.API(auth, wait_on_rate_limit= True)

In [40]:
# extract 200 tweets from twitter user
posts = api.user_timeline(screen_name = "MutahiNgunyi", count=200, lang="en", tweet_mode="extended")

i = 1
for tweet in posts[0:5]:
  print(str(i)+ ")" + tweet.full_text + "\n")
  i+=1
# print last 5 twets from account
# for tweet in posts[0:5]:
#   print(tweet.full_text + "\n")

1)FACT: I am the BIOLOGICAL father of the Hustler Nation, Ruto is the SURROGATE father. But NOW my creation has turned into a FELLOWSHIP of THUGS #UDANDC

2)Dear William Ruto: If you BELIEVE what you have said at the #UDANDC regarding the FAILURES of Uhuru Kenyatta, you should RESIGN. Otherwise the RECORD must REFLECT that you are LYING. Do not INSULT our collective intelligence.

3)Dear Kiambu. Also KNOWN as Kiambaa. Our HEARTS have to come to TERMS with this. I hate this VIDEO. Especially the part of a child trying to ESCAPE Kiambaa church and is thrown BACK into the FIRE. Heartless. https://t.co/dc0oMH2qoh

4)QUESTION: Is it REMOTELY possible for Raila to go to KERICHO, abuse Ruto and Kalenjins CHEER him wildly? And can Ruto ABUSE Raila in Kisumu with Luos CHEERING? Ruto just did that in Uhuru's backyard and kikuyus CHEERED him ON. Either they have LOST it or they are SETTING him UP!!

5)Dear William Ruto: GEMA will betray YOU. It is not a matter of WHETHER or NOT. It is a matter of

In [41]:
# create a data frame witha column called Tweets
df = pd.DataFrame([tweet.full_text for tweet in posts], columns=["Tweets"])

df.head(5)

,Tweets
0,FACT: I am the BIOLOGICAL father of the Hustle...
1,Dear William Ruto: If you BELIEVE what you hav...
2,Dear Kiambu. Also KNOWN as Kiambaa. Our HEARTS...
3,QUESTION: Is it REMOTELY possible for Raila to...
4,Dear William Ruto: GEMA will betray YOU. It is...


# Text Preprocessing Techniques

## Remove Punctuations
Removing punctuation from the text like ".?!" and also the symbols like "@#$" .

In [42]:
# create a function to clean the tweets
def cleanText(text):
  text = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", '', text) # Remoove the hyperlink
  return text

df['Tweets'] = df['Tweets'].apply(cleanText)
df.head(10)

,Tweets
0,FACT I am the BIOLOGICAL father of the Hustler...
1,Dear William Ruto If you BELIEVE what you have...
2,Dear Kiambu Also KNOWN as Kiambaa Our HEARTS h...
3,QUESTION Is it REMOTELY possible for Raila to ...
4,Dear William Ruto GEMA will betray YOU It is n...
5,There was an EARTHQUAKE It was YESTERDAY And i...
6,Dear Kikuyus I do not SEE how UDA will beat AZ...
7,HATE SPEECH commission NCIC is a committee of ...
8,Boda Boda INDUSTRY is a chaotic AGGREGATE Ruto...
9,Should Kalonzo join Azimio or UDA Whichever WA...


In [43]:
contractions_dict = {     
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I had",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "iit will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

## Expanding Contractions

Contraction helps to expand the word form like "ain't": "am not". 

Contractions file has been created in my github which we will be importing to use it.

In [44]:
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(text, contractions_dict=contractions_dict):
     def replace(match):
         return contractions_dict[match.group(0)]
     return contractions_re.sub(replace, text)

df['Tweets'] = df['Tweets'].apply(expand_contractions)
df.head(5)

,Tweets
0,FACT I am the BIOLOGICAL father of the Hustler...
1,Dear William Ruto If you BELIEVE what you have...
2,Dear Kiambu Also KNOWN as Kiambaa Our HEARTS h...
3,QUESTION Is it REMOTELY possible for Raila to ...
4,Dear William Ruto GEMA will betray YOU It is n...


## Lemmatizing

Lemmatize the text so as to get its root form 
eg: functions,funtionality as function

In [45]:
lemmatizer = WordNetLemmatizer()
tokenizer = nltk.tokenize.WhitespaceTokenizer()

def lemmatize_text(text):
  """
  take string input and lemmatize the words.
  use WordNetLemmatizer to lemmatize the words.
  """
  lemmatized_word = [lemmatizer.lemmatize(w) for w in tokenizer.tokenize(text)]
  return (" ".join(lemmatized_word))


df['Tweets'] = df['Tweets'].apply(lemmatize_text)
df.head(5)


,Tweets
0,FACT I am the BIOLOGICAL father of the Hustler...
1,Dear William Ruto If you BELIEVE what you have...
2,Dear Kiambu Also KNOWN a Kiambaa Our HEARTS ha...
3,QUESTION Is it REMOTELY possible for Raila to ...
4,Dear William Ruto GEMA will betray YOU It is n...


## Remove numbers
Removing numbers from the text like "1,2,3,4,5…" 

We usually remove numbers when we do text clustering or getting keyphrases as we numbers doesn't give much importance to get the main words. 

In [46]:
def remove_numbers(text):
  """
  take string input and return a clean text without numbers. 
  Use regex to discard the numbers.
  """
  text = re.sub(r"^\d+\s|\s\d+\s|\s\d+$", " ", text)
  return text

df['Tweets'] = df['Tweets'].apply(remove_numbers)
df.head(5)

,Tweets
0,FACT I am the BIOLOGICAL father of the Hustler...
1,Dear William Ruto If you BELIEVE what you have...
2,Dear Kiambu Also KNOWN a Kiambaa Our HEARTS ha...
3,QUESTION Is it REMOTELY possible for Raila to ...
4,Dear William Ruto GEMA will betray YOU It is n...


## Word Count
We can perform a word count to check which are the most used words in our data.

In [47]:
from collections import Counter
results = Counter()

df['Tweets'].str.lower().str.split().apply(results.update)

word_count = pd.DataFrame(results.items(), columns=['Word', 'wordCount'])

word_count.head(10)

,Word,wordCount
0,fact,14
1,i,39
2,am,10
3,the,253
4,biological,1
5,father,3
6,of,122
7,hustler,10
8,nation,9
9,ruto,80


## Stop words removal

Remove irrelevant words using nltk stop words like "is,the,a" etc from the sentences as they don't carry any information.

In [48]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(sentence):
    """
    removes all the stop words like "is,the,a, etc."
    5 lines of code can be written in one line as:
        return ' '.join([w for w in word_tokenize(sentence) if not w in stop_words]) 
    """
    clean_sent =[]
    for w in word_tokenize(sentence):
        if not w in stop_words:
            clean_sent.append(w)
    return " ".join(clean_sent)

df['Tweet'] = df['Tweets'].apply(remove_stopwords)
df.head(5)

,Tweets,Tweet
0,FACT I am the BIOLOGICAL father of the Hustler...,FACT I BIOLOGICAL father Hustler Nation Ruto S...
1,Dear William Ruto If you BELIEVE what you have...,Dear William Ruto If BELIEVE said UDANDC regar...
2,Dear Kiambu Also KNOWN a Kiambaa Our HEARTS ha...,Dear Kiambu Also KNOWN Kiambaa Our HEARTS come...
3,QUESTION Is it REMOTELY possible for Raila to ...,QUESTION Is REMOTELY possible Raila go KERICHO...
4,Dear William Ruto GEMA will betray YOU It is n...,Dear William Ruto GEMA betray YOU It matter WH...


Having created a new column with the a clean data that the stop words have been removed

We can drop the tweets column and remain with only one column containing the data.

In [49]:
df = df.drop('Tweets', 1)

# now when we check our dataframe we will only have one column of data to work with
df.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Tweet
0,FACT I BIOLOGICAL father Hustler Nation Ruto S...
1,Dear William Ruto If BELIEVE said UDANDC regar...
2,Dear Kiambu Also KNOWN Kiambaa Our HEARTS come...
3,QUESTION Is REMOTELY possible Raila go KERICHO...
4,Dear William Ruto GEMA betray YOU It matter WH...


# Word Count

After removing the stop words. Lets perform a word counter Again

In [50]:
from collections import Counter
results = Counter()

df['Tweet'].str.lower().str.split().apply(results.update)

word_count = pd.DataFrame(results.items(), columns=['Word', 'wordCount'])

word_count.head(10)

,Word,wordCount
0,fact,14
1,i,39
2,biological,1
3,father,3
4,hustler,10
5,nation,9
6,ruto,80
7,surrogate,1
8,but,14
9,now,11


## Convert Letters to lower
Converting text to lower case as in, converting "Question" to "question" or "QUESTION" to "question".

In [52]:
def to_lower(text):
    """
    Converting text to lower case as in, 
    converting "Question" to "question" or "QUESTION" to "question".
    """
    return ' '.join([tweet.lower() for tweet in word_tokenize(text)])


df['Tweet'] = df['Tweet'].apply(to_lower)

df.head(5)


,Tweet
0,fact i biological father hustler nation ruto s...
1,dear william ruto if believe said udandc regar...
2,dear kiambu also known kiambaa our hearts come...
3,question is remotely possible raila go kericho...
4,dear william ruto gema betray you it matter wh...


## Spell Check

correct the incorrect spelled words like 'hme' to 'home'

**"Our data is full of slang words which will make this library not work as it will give a false transalation to most of our data.

You can omit this text processing" **

In [53]:
# spell = Speller(lang='en')
# def autospell(text):
#     """
#     correct the spelling of the word.
#     """
#     spells = [spell(w) for w in (nltk.word_tokenize(text))]
#     return " ".join(spells)

# df['Tweet'] = df['Tweet'].apply(autospell)
# df['Tweets'] = df['Tweets'].apply(autospell)
# df.head(5)